In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/kcah/Documents/code-repo/air-quality-prediction
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) 

location_str

2025-11-17 21:45:17,510 INFO: Initializing external client
2025-11-17 21:45:17,510 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-17 21:45:19,072 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279136


'{"country": "finland", "city": "helsinki", "street": "kluuvi", "aqicn_url": "https://api.waqi.info/feed/@5717", "latitude": "60.1733244", "longitude": "24.9410248"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_lag',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_extended',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,18.0,finland,helsinki,kluuvi,2025-11-17,https://api.waqi.info/feed/@5717


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[us]
 5   url      1 non-null      object        
dtypes: datetime64[us](1), float32(1), object(4)
memory usage: 176.0+ bytes


In [7]:
# import datetime
# import pandas as pd

# # Convert string dates to date objects
# date_start = datetime.date(2025, 11, 12)
# date_end = datetime.date(2025, 11, 16)

# # Read data for both dates
# historical_data = air_quality_fg.filter(
#     (air_quality_fg.country == country) &
#     (air_quality_fg.city == city) &
#     (air_quality_fg.street == street) &
#     (air_quality_fg.date >= date_start) &
#     (air_quality_fg.date <= date_end)
# ).read()

# # Sort by date
# historical_data = historical_data.sort_values('date')
# historical_data

## Add Lagged Air Quality Features

We will retrieve the previous 1, 2, and 3 days of air quality data from the feature store and add them as lagged features.


In [8]:
# Retrieve previous days' air quality data to create lagged features
from datetime import timedelta
from datetime import timezone

# Get dates for 1, 2, and 3 days ago, set to midnight and normalize to UTC
today = pd.Timestamp.now(tz='UTC')
date_1d_ago = (today - timedelta(days=1)).astimezone(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
date_2d_ago = (today - timedelta(days=2)).astimezone(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
date_3d_ago = (today - timedelta(days=3)).astimezone(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
date_4d_ago = (today - timedelta(days=4)).astimezone(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

# Initialize lagged features as None
aq_today_df['pm25_lag_1d'] = None
aq_today_df['pm25_lag_2d'] = None
aq_today_df['pm25_lag_3d'] = None

# Try to retrieve previous days' data from the feature store
try:
    # Query for previous days' pm25 values
    historical_data = air_quality_fg.filter(
        (air_quality_fg.country == country) &
        (air_quality_fg.city == city) &
        (air_quality_fg.street == street) &
        (air_quality_fg.date >= date_4d_ago) &
        (air_quality_fg.date < today)
    ).read()
    
    # Select only the columns we need
    historical_data = historical_data[['date', 'pm25']]
    
    # Sort by date
    historical_data = historical_data.sort_values('date')
    
    # Normalize historical_data dates to UTC to match comparison timestamps
    # This handles the Etc/UTC vs UTC timezone difference
    if historical_data['date'].dt.tz is not None:
        historical_data['date'] = historical_data['date'].dt.tz_convert('UTC')
    
    # Get the pm25 values for 1, 2, and 3 days ago
    for idx, row in aq_today_df.iterrows():
        # Convert dates to pandas Timestamps and normalize to UTC for comparison
        ts_1d_ago = pd.Timestamp(date_1d_ago).tz_convert('UTC')
        ts_2d_ago = pd.Timestamp(date_2d_ago).tz_convert('UTC')
        ts_3d_ago = pd.Timestamp(date_3d_ago).tz_convert('UTC')
        
        # Get pm25 for 1 day ago
        lag_1d = historical_data[historical_data['date'] == ts_1d_ago]
        if not lag_1d.empty:
            aq_today_df.at[idx, 'pm25_lag_1d'] = lag_1d['pm25'].iloc[0]
        
        # Get pm25 for 2 days ago
        lag_2d = historical_data[historical_data['date'] == ts_2d_ago]
        if not lag_2d.empty:
            aq_today_df.at[idx, 'pm25_lag_2d'] = lag_2d['pm25'].iloc[0]
        
        lag_3d = historical_data[historical_data['date'] == date_3d_ago]
        if not lag_3d.empty:
            aq_today_df.at[idx, 'pm25_lag_3d'] = lag_3d['pm25'].iloc[0]

except Exception as e:
    print(f"Warning: Could not retrieve lagged features: {e}")
    print("Lagged features will be set to None for this day")
    raise e

# Convert lagged features to float32, handling None values
aq_today_df['pm25_lag_1d'] = pd.to_numeric(aq_today_df['pm25_lag_1d'], errors='coerce').astype('float32')
aq_today_df['pm25_lag_2d'] = pd.to_numeric(aq_today_df['pm25_lag_2d'], errors='coerce').astype('float32')
aq_today_df['pm25_lag_3d'] = pd.to_numeric(aq_today_df['pm25_lag_3d'], errors='coerce').astype('float32')

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.66s) 


,pm25,country,city,street,date,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
0,18.0,finland,helsinki,kluuvi,2025-11-17,https://api.waqi.info/feed/@5717,9.0,13.0,13.0


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
start_date = pd.Timestamp.today().normalize()
end_date = start_date + pd.Timedelta(days=10)
# Convert dates to 'YYYY-MM-DD' format strings
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude, start_date, end_date)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 60.25°N 25.0°E
Elevation 2.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,rain_sum,city
0,2025-11-17,0.65,0.0,8.209263,344.744812,0.00,helsinki
1,2025-11-18,-2.10,0.0,11.269782,333.435028,0.00,helsinki
2,2025-11-19,-1.65,0.0,7.903619,59.931511,0.00,helsinki
3,2025-11-20,2.70,0.1,20.929596,183.945114,0.10,helsinki
4,2025-11-21,0.40,0.6,14.561099,351.469299,0.30,helsinki
5,2025-11-22,3.50,0.0,19.966211,244.359039,0.00,helsinki
6,2025-11-23,5.00,0.0,12.069400,197.354111,0.00,helsinki
7,2025-11-24,2.60,0.5,13.493999,189.210953,0.50,helsinki
8,2025-11-25,3.55,0.2,6.369050,317.290619,0.20,helsinki
9,2025-11-26,0.35,0.5,17.884428,40.100849,0.25,helsinki


In [13]:
hourly_df

,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,rain_sum
date,,,,,
2025-11-17 00:00:00,2.40,0.2,8.891343,238.240555,0.2
2025-11-17 01:00:00,1.80,0.0,6.618519,247.619812,0.0
2025-11-17 02:00:00,0.95,0.0,4.320000,270.000000,0.0
2025-11-17 03:00:00,0.40,0.0,3.600000,306.869965,0.0
2025-11-17 04:00:00,0.35,0.0,4.510787,331.389618,0.0
...,...,...,...,...,...
2025-11-27 19:00:00,-0.65,0.0,11.620809,16.189287,0.0
2025-11-27 20:00:00,-0.70,0.0,10.587917,17.818983,0.0
2025-11-27 21:00:00,-0.70,0.0,9.904906,19.093594,0.0


In [14]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         11 non-null     datetime64[ns]
 1   temperature_2m_mean          11 non-null     float32       
 2   precipitation_sum            11 non-null     float32       
 3   wind_speed_10m_max           11 non-null     float32       
 4   wind_direction_10m_dominant  11 non-null     float32       
 5   rain_sum                     11 non-null     float32       
 6   city                         11 non-null     object        
dtypes: datetime64[ns](1), float32(5), object(1)
memory usage: 528.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [15]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-17 21:52:22,322 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279136/fs/1265746/fg/1721860


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_lag_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279136/jobs/named/air_quality_lag_1_offline_fg_materialization/executions


(Job('air_quality_lag_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762063
         }
       },
       "result": {
         "observed_value": 18.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T08:52:22.000322Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expe

In [16]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-17 21:52:38,678 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279136/fs/1265746/fg/1718851


Uploading Dataframe: 100.00% |██████████| Rows 11/11 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_extended_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279136/jobs/named/weather_extended_1_offline_fg_materialization/executions
2025-11-17 21:52:55,775 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 21:52:58,949 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 21:54:40,643 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 21:54:40,804 INFO: Waiting for log aggregation to finish.
2025-11-17 21:55:03,121 INFO: Execution finished successfully.


(Job('weather_extended_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 758980
         }
       },
       "result": {
         "observed_value": 6.369049549102783,
         "element_count": 11,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T08:52:38.000678Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectati

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
